In [29]:
#Q1

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from pytorch_lightning import LightningModule
import random

class CNN(LightningModule):
    def __init__(self,in_channels,num_filters_conv, filter_sizes_conv, num_filters_dense ,activation='relu' ):
        super(CNN, self).__init__()
        
        # Select the activation function
        if activation == 'relu':
            self.acitvation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'gelu':
            self.activation = nn.GELU()
        elif activation == 'elu':
            self.activation  = nn.ELU()
        else:
            self.activation = nn.SiLU()
            
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=num_filters_conv[0], kernel_size=filter_sizes_conv[0],stride=1, padding=1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=num_filters_conv[0], out_channels=num_filters_conv[1], kernel_size=filter_sizes_conv[1],stride =1 , padding=1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=num_filters_conv[1], out_channels=num_filters_conv[2], kernel_size=filter_sizes_conv[2],stride=1, padding=1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=num_filters_conv[2], out_channels=num_filters_conv[3], kernel_size=filter_sizes_conv[3],stride=1, padding=1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=num_filters_conv[3], out_channels=num_filters_conv[4], kernel_size=filter_sizes_conv[4],stride=1, padding=1),
            self.activation,
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.dense_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=num_filters_conv[-1]*3*3, out_features=10),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.dense_layers(x)
        return x

# Assuming the use of PyTorch Lightning for training setup
class LitCNN(CNN):
    def __init__(self):
        super().__init__()

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer
in_channels = 3  # For RGB images
num_filters_conv=[32, 64, 128, 256, 512]
filter_sizes_conv=[3, 3, 3, 3, 3]
num_filters_dense=[1024]
activation = random.choice(['relu','tanh','sigmoid','gelu', 'elu', 'silu'])
model = CNN(in_channels,num_filters_conv, filter_sizes_conv,num_filters_dense,activation)
print(model)
# Note: You'll need to define your data loaders and training loop.


CNN(
  (activation): GELU(approximate='none')
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GELU(approximate='none')
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): GELU(approximate='none')
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): GELU(approximate='none')
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): GELU(approximate='none')
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): GELU(approximate='none')
    (14): MaxPool2d(kernel_size=2, s

In [ ]:
#Q2

In [1]:
!pip install --upgrade wandb
import cv2
import glob
import random
torch.manual_seed(7)
torch.cuda.empty_cache()
from torch import  nn,optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from pandas.core.common import flatten
from tqdm import tqdm 
import torchvision.transforms as T
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import wandb
from wandb.keras import WandbCallback
import torch
import torchvision

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://wandb.ai/" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -c -O 'nature_12K.zip'

In [34]:
!unzip "nature_12K.zip"

cpu


In [2]:
import os
def check_os():
    if os.name == 'nt':
        return 'Windows'
    else:
        return 'Linux'
operatingSystem = check_os()
# print(operatingSystem)

Windows


In [3]:
#Custom dataset class for inaturalist dataset
class iNaturalist(Dataset):
    def __init__(self, image_paths, class_to_idx, transform):
        self.image_paths = image_paths
        self.transform = transform
        self.class_to_idx= class_to_idx
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.cvtColor(cv2.imread(image_filepath), cv2.COLOR_BGR2RGB)
        # print(image_filepath)
        if(operatingSystem=='Windows'):
            label = image_filepath.split('\\')[-2]
        else:
            label = image_filepath.split('/')[-2]
        # print(label)
        label = self.class_to_idx[label]
        
        PIL_image = Image.fromarray(np.uint8(image)).convert('RGB')
        PIL_image = Image.fromarray(image.astype('uint8'), 'RGB')
        PIL_image = self.transform(PIL_image)

        return PIL_image, label

In [4]:
def forwardSlash(l):
    a=[]
    for ele in l:
        a.append(ele.replace('\\','/'))
    return a
    
def create_data(data_type, data_path,  data_aug, image_shape, b_size):
    #Defining transformations when data_aug=True  [used when data_type='train' and data_aug=True]
    if(data_aug):
        transforms = T.Compose([T.Resize((image_shape)),
                              T.RandomRotation(degrees=15),
                              T.RandomHorizontalFlip(p=0.5),
                              T.RandomGrayscale(p=0.2),
                              T.ToTensor()])
    else:
    #Defining transformations when data_aug=False
        transforms = T.Compose([T.Resize((image_shape)),
                               T.ToTensor()])
    image_paths=[] # List to store image paths
    classes= [] # List to  store class values
    #get all the paths from data_path and append image paths and class to to respective lists
    cnt=0
    for curr_data_path in glob.glob(data_path + '/*'):
        if(operatingSystem=='Windows'):
            classes.append(curr_data_path.split('\\')[-1])
        else:
            classes.append(curr_data_path.split('/')[-1])
        image_paths.append(glob.glob(curr_data_path+'/*'))
    image_paths = list(flatten(image_paths))
    # image_paths = forwardSlash(image_paths)
    #Creating dictionary for class indexes
    idx_to_class={}
    class_to_idx={}
    for i in range(len(classes)):
        idx=i
        cls=classes[i]
        idx_to_class[idx]=cls
        class_to_idx[cls]=idx
    # idx_to_class = {i:j for i, j in enumerate(classes)}
    # for k in idx_to_class:
        # v=idx_to_class[k]
        # class_to_idx[v]=k
    # class_to_idx = {value:key for key,value in idx_to_class.items()}
    if (data_type != 'test'):
        random.shuffle(image_paths)
        # 80% training data and 20% validation data
        train_image_paths = image_paths[:int(0.8*len(image_paths))]
        valid_image_paths = image_paths[int(0.8*len(image_paths)):] 
        #Using custom class for getting train and validation dataset
        # if data_aug == True:
        train_dataset = iNaturalist(train_image_paths,class_to_idx,transforms)
        validation_dataset = iNaturalist(valid_image_paths,class_to_idx,transforms)  
        # else:
        #     train_dataset = iNaturalist(train_image_paths,class_to_idx,false_transforms)
        #     valid_dataset = iNaturalist(valid_image_paths,class_to_idx,false_transforms)  
        #using Dataloader to load train and valid dataset according to batch size
        train_loader = DataLoader(train_dataset, batch_size=b_size, shuffle=True)
        validation_loader = DataLoader(validation_dataset, batch_size=b_size, shuffle=True)
        return train_loader,validation_loader
    else:
        # test_image_paths=image_paths
        #Using custom class for getting test dataset
        transforms = T.Compose([T.Resize((image_shape)),
                               T.ToTensor()])
        test_dataset= iNaturalist(image_paths,class_to_idx,transforms)
        #using Dataloader to load test dataset according to batch size
        test_loader = DataLoader(test_dataset, batch_size=b_size, shuffle=True)
        return test_loader
    

In [5]:
from pytorch_lightning import LightningModule
class ConvolutionBlocks(LightningModule):
    def __init__(self, activation, batch_norm, size_filters, filter_organization, number_filters):
        super(ConvolutionBlocks, self).__init__()
        
        self.activation=activation
        self.num_filters=[number_filters]
        self.batch_norm=batch_norm
        
        for i in range(1,5):
          self.num_filters.append(int(self.num_filters[i-1]*filter_organization))
            
        if(self.batch_norm):  
            self.conv_layers = nn.Sequential(
                nn.Conv2d(in_channels=3,out_channels=self.num_filters[0],kernel_size=size_filters[0],stride=(1, 1),padding=(1, 1),bias=False),
                nn.BatchNorm2d(self.num_filters[0]),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                
                nn.Conv2d(in_channels=self.num_filters[0],out_channels=self.num_filters[1],kernel_size=size_filters[1],stride=(1, 1),padding=(1, 1),bias=False),
                nn.BatchNorm2d(self.num_filters[1]),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[1],out_channels=self.num_filters[2],kernel_size=size_filters[2],stride=(1, 1),padding=(1, 1),bias=False),
                nn.BatchNorm2d(self.num_filters[2]),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[2],out_channels=self.num_filters[3],kernel_size=size_filters[3],stride=(1, 1),padding=(1, 1),bias=False),
                nn.BatchNorm2d(self.num_filters[3]),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[3],out_channels=self.num_filters[4],kernel_size=size_filters[4],stride=(1, 1),padding=(1, 1),bias=False),
                nn.BatchNorm2d(self.num_filters[4]),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
            )
        else:
            self.conv_layers = nn.Sequential(
                nn.Conv2d(in_channels=3,out_channels=self.num_filters[0],kernel_size=size_filters[0],stride=(1, 1),padding=(1, 1),bias=False),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
                
                nn.Conv2d(in_channels=self.num_filters[0],out_channels=self.num_filters[1],kernel_size=size_filters[1],stride=(1, 1),padding=(1, 1),bias=False),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[1],out_channels=self.num_filters[2],kernel_size=size_filters[2],stride=(1, 1),padding=(1, 1),bias=False),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[2],out_channels=self.num_filters[3],kernel_size=size_filters[3],stride=(1, 1),padding=(1, 1),bias=False),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
    
                nn.Conv2d(in_channels=self.num_filters[3],out_channels=self.num_filters[4],kernel_size=size_filters[4],stride=(1, 1),padding=(1, 1),bias=False),
                self.activation,
                nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
            )
            

    def forward(self, x):
        return self.conv_layers(x)
     


In [6]:
class Model(nn.Module):
    def __init__(self, number_initial_filters , neurons_in_dense_layer, image_shape,dropout , activation, batch_norm, size_filters, filter_organization):
        super().__init__()
        
        if(activation=='relu'):
            self.activation = nn.ReLU()
        elif(activation=='gelu'):
            self.activation = nn.GELU()
        elif(activation=='silu'):
            self.activation = nn.SiLU()
        else:
            self.activation = nn.Mish()
            

        self.conv_blocks = ConvolutionBlocks(self.activation, batch_norm, size_filters, filter_organization, number_initial_filters)
        sz=self.conv_blocks(torch.zeros(*(image_shape))).data.shape
        fc1_in_channels = sz[1] * sz[2] * sz[3]
        self.output= nn.Linear(neurons_in_dense_layer,10,bias=True)   

        self.dense_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(fc1_in_channels,neurons_in_dense_layer,bias=True),
            self.activation,
            nn.Dropout(p=dropout)
        )
        
    def forward(self, x):
        x = self.conv_blocks(x)
        x = self.dense_layers(x)
        x = F.softmax(self.output(x),dim=1) #Applying softmax across rows
        return x

In [7]:
def cal(data,device,model,criterion):
    loss=0
    corr_sample=0
    tot_sample=0
    for x,y in tqdm(data,total=len(data)):
        x=x.to(device=device)
        y=y.to(device=device)
        out = model(x)
        loss+=criterion(out,y).item()
        _,pred=out.max(1)
        corr_sample+=(pred==y).sum().item()
        tot_sample+=pred.size(0)
    return corr_sample,tot_sample,loss
    
def evaluate(device, loader, model):
    ''' Function to calculate accuracy to see performance of our model '''
    
    model.eval()
    criterion = nn.CrossEntropyLoss()
    correct_samples,total_samples,loss=0,0,0
    with torch.no_grad():
        correct_samples_,total_samples_,loss_ = cal(loader,device,model,criterion)
        correct_samples+=correct_samples_
        total_samples+=total_samples_
        loss+=loss_
           
    acc = round((correct_samples / total_samples) * 100, 4)
    return acc, loss/total_samples 
 

In [8]:
def cal_train(data,device,model,criterion,optimizer):
    train_loss=0
    corr_train=0
    tot_sample=0
    for idd,(x,y) in enumerate(tqdm(data)):
        x=x.to(device=device)
        yt=y.to(device=device)
        out = model(x)
        loss=criterion(out,yt)
        train_loss+=loss.item()
        _,pred=out.max(1)
        corr_train+=(pred==yt).sum()
        tot_sample+=pred.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return train_loss,tot_sample,corr_train


def train(args):
    torch.cuda.empty_cache()
    image_shape = (1,3,224,224) # All the images of dataset will get resized to this image shape
    test_data_path = 'inaturalist_12K/val/'
    train_data_path = 'inaturalist_12K/train/'
     
    wandb.run.name = 'ep-'+str(args.epochs)+'-lr-'+str(args.learning_rate)+'-bs-'+str(args.batch_size)+'-act-'+str(args.activation)+'-filter_sizes-'+str(args.size_filters) +'-ini_filt'+str(args.number_initial_filters)+'-n_d-'+str(args.neurons_in_dense_layer)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #CNN Model creation
    model = Model(args.number_initial_filters ,args.neurons_in_dense_layer,image_shape,args.dropout, args.activation, args.batch_norm, args.size_filters, args.filter_organization).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    # Train Network
    for epoch in range(args.epochs):
        model.train()
        test_loader = create_data("test",test_data_path,args.data_aug, image_shape[2:], args.batch_size)
        train_loader, valid_loader = create_data("train",train_data_path,args.data_aug,image_shape[2:], args.batch_size)
        
        train_loss,total_samples,train_correct = cal_train(train_loader,device,model,criterion,optimizer)
        # Calculating training accuracy and training loss
        train_loss /= total_samples
        train_acc = round((train_correct / total_samples).item()  * 100, 4)
        
       
        
        # Calculating accuracy and loss for test and validataion data
        val_acc, val_loss = evaluate(device, valid_loader, model)
        test_acc, test_loss = evaluate(device, test_loader, model)

        
        #logging wandb data
        wandb.log(
          {'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc, 'train_loss': train_loss, 'val_loss': val_loss, 'test_loss': test_loss}
        )
        print('\nEpoch ', epoch, 'train_acc', train_acc, 'val_acc', val_acc, 'test_acc', test_acc, 'train_loss', train_loss, 'val_loss', val_loss, 'test_loss', test_loss) 

In [9]:
wandb.login(key = "67fcf10073b0d1bfeee44a1e4bd6f3eb5b674f8e")
sweep_config = {
    "name" : "Assignment2_local",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "batch_norm": {
              "values": [True,False]
        },
        "neurons_in_dense_layer": {
              "values": [32, 64, 128, 256, 512, 1024]
          },
        "epochs" : {
            "values" : [5,8,10]
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        "data_aug": {
              "values": [True,False]
        },
        'size_filters':{
            'values': [[7,5,5,3,3], [11,9,7,5,3]]
        },
        'filter_organization': {
            'values': [1, 2, 0.5]
        },
        'number_initial_filters': {
            'values': [16, 32, 64, 128]
        },
        'activation': {
            'values': ['relu', 'mish', 'gelu', 'silu]
        },
        'learning_rate':{
            "values": [0.001,0.0001,0.0003,0.0005]
        },
        "dropout": {
            "values": [0,0.1,0.2,0.3]
        }        
    }
}
sweep_id = wandb.sweep(sweep_config, project="Assigment2_local", entity="cs23m055")
print('sweep_id: ', sweep_id)

wandb: Currently logged in as: cs23m055 (cs23m055_assignment_1). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ravis\.netrc


Create sweep with ID: uxe0iw1a
Sweep URL: https://wandb.ai/cs23m055/Assigment2_local/sweeps/uxe0iw1a
sweep_id:  uxe0iw1a


In [10]:
def spawn_fn():
      with wandb.init(project="Assignment2_local", entity="cs23m055") as run:
        config = wandb.config
        train(config)

In [11]:
wandb.agent(sweep_id, function=spawn_fn, count=1)

wandb: Agent Starting Run: e5iw86xj with config:
wandb: 	activation: mish
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_aug: False
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	filter_organization: 1
wandb: 	learning_rate: 0.0001
wandb: 	neurons_in_dense_layer: 1024
wandb: 	number_initial_filters: 128
wandb: 	size_filters: [7, 5, 5, 3, 3]
wandb: Currently logged in as: cs23m055. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [01:17<00:00,  1.23s/it]



Epoch  0 train_acc 23.2529 val_acc 23.65 test_acc 25.65 train_loss 0.06919991944727115 val_loss 0.06953782236576081 test_loss 0.06907254886627197


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [01:18<00:00,  1.25s/it]



Epoch  1 train_acc 27.3159 val_acc 24.45 test_acc 24.9 train_loss 0.06803040132476086 val_loss 0.06937076210975647 test_loss 0.06924087285995484


...

100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [01:20<00:00,  1.27s/it]



Epoch  3 train_acc 31.3664 val_acc 32.2 test_acc 28.7 train_loss 0.06682003758999538 val_loss 0.06714371037483215 test_loss 0.068160549223423


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [01:23<00:00,  1.33s/it]



Epoch  4 train_acc 32.4666 val_acc 32.25 test_acc 31.2 train_loss 0.06645569030546161 val_loss 0.06691086000204086 test_loss 0.06742456823587417


test_acc,▂▁▇▅█
test_loss,▇█▂▄▁
train_acc,▁▄▆▇█
train_loss,█▅▃▂▁
val_acc,▁▂▅██
val_loss,██▅▂▁
test_acc,31.2
test_loss,0.06742
train_acc,32.4666
train_loss,0.06646
val_acc,32.25
